# Predictive Modeling 

# Defining ANN Architecture

In this notebook, we will define the architecture of the Artificial Neural Network (ANN) model as part of the predictive modeling process. This task involves setting up the input, hidden, and output layers, selecting the appropriate activation functions, and defining the optimization algorithm.


## Loading Configuration

In this step, we load the configuration file that contains the paths to the datasets and directories needed for the predictive modeling task. The `config.json` file, located in the root directory, is accessed and loaded into the environment to provide the necessary paths for the data processing steps.

This configuration file is crucial as it allows us to dynamically change paths without hardcoding them, making the code more flexible and easier to manage.


In [17]:
import os
import json

# Determine the base directory (the root of your project)
base_dir = os.path.dirname(os.path.dirname(os.path.abspath('')))

# Load the configuration file from the main directory
config_path = os.path.join(base_dir, 'config.json')
with open(config_path) as config_file:
    config = json.load(config_file)

# Display the loaded configuration for verification
config


{'raw_data_path': 'data/raw/Dataset (ATS)-1.csv',
 'interim_cleaned_data_path': 'data/interim/cleaned_dataset.csv',
 'preprocessed_data_path': 'Data_Preparation/preprocessed_dataset/cleaned_dataset.csv',
 'processed_data_path': 'data/processed/processed_dataset_with_features.csv',
 'train_data_path': 'data/train/train_dataset.csv',
 'test_data_path': 'data/test/test_dataset.csv',
 'min_max_scaled_path': 'Data_Preparation/scaling_techniques/min_max_scaled_dataset.csv',
 'standard_scaled_path': 'Data_Preparation/scaling_techniques/standard_scaled_dataset.csv',
 'training_set_path': 'Data_Preparation/training_sets/train_dataset.csv',
 'testing_set_path': 'Data_Preparation/testing_sets/test_dataset.csv',
 'min-max_scaled_4_clusters_path': 'Clustering_Analysis/kmeans_model/min-max_scaled_4_clusters.csv',
 'standard_scaled_4_clusters_path': 'Clustering_Analysis/kmeans_model/standard_scaled_4_clusters.csv',
 'min-max_scaled_cluster_characteristics_path': 'Clustering_Analysis/kmeans_model/min-

## Data Loading

Next, we load the training and testing datasets as specified in the configuration file. The training dataset will be used to train the model, and the testing dataset will be used to evaluate the model's performance.

It's important to ensure that the data is loaded correctly and matches the structure expected by the model, which includes features (X) and the target variable (y) for both training and testing data.


In [18]:
import pandas as pd # type: ignore

# Define paths using absolute paths
train_data_path = os.path.join(base_dir, config['train_data_path'])
test_data_path = os.path.join(base_dir, config['test_data_path'])

# Load the training and testing data
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

# Separate features (X) and target (y) for both training and testing data
X_train = train_data.drop(columns=['Churn_No', 'Churn_Yes'])
y_train = train_data['Churn_Yes']

X_test = test_data.drop(columns=['Churn_No', 'Churn_Yes'])
y_test = test_data['Churn_Yes']


## Defining the ANN Architecture

In this step, we define the architecture of the Artificial Neural Network (ANN). The model structure includes:

- **Input Layer**: Accepts input features.
- **Hidden Layers**: Performs computations and learning.
- **Output Layer**: Provides the final prediction.

We also specify the activation functions (`relu` for hidden layers and `sigmoid` for the output layer) and the optimizer (`Adam`) to use during training.

The architecture is crucial for determining the model's ability to learn patterns in the data and make accurate predictions.


### Model Training and Validation

In this step, we train the ANN model using the training data. During training, the model learns the relationships between the input features and the target variable. We use a validation set (testing data) to monitor the model's performance and adjust the training process accordingly.

To prevent overfitting, we use techniques such as dropout layers and model checkpoints. The best model is saved based on validation performance, ensuring that we capture the model with the best generalization capabilities.


In [19]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Dropout # type: ignore # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore # type: ignore
from tensorflow.keras.callbacks import ModelCheckpoint # type: ignore

# Define the architecture of the ANN model
model = Sequential()

# Input layer
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))

# Hidden layers
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=16, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Define the model checkpoint to save the best model
trained_model_path = os.path.join(base_dir, config['trained_model_path'])
checkpoint = ModelCheckpoint(os.path.join(trained_model_path, 'best_model.h5'), 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_test, y_test), 
                    callbacks=[checkpoint])

# Save the model architecture
ann_architecture_path = os.path.join(base_dir, config['ann_architecture_path'])
with open(os.path.join(ann_architecture_path, 'ann_architecture.json'), 'w') as f:
    f.write(model.to_json())

# Save training results
results_path = os.path.join(base_dir, config['results_path'])
with open(os.path.join(results_path, 'training_results.txt'), 'w') as f:
    f.write(str(history.history))

print("Training complete. Model and results saved.")


Epoch 1/50
169/177 [===========================>..] - ETA: 0s - loss: 9.0644 - accuracy: 0.6562 
Epoch 1: val_loss improved from inf to 0.58182, saving model to d:\Customer-Churn-Analysis\Predictive_Modeling/trained_model\best_model.h5
177/177 [==============================] - 1s 4ms/step - loss: 8.8019 - accuracy: 0.6578 - val_loss: 0.5818 - val_accuracy: 0.7353
Epoch 2/50
 76/177 [===========>..................] - ETA: 0s - loss: 1.6363 - accuracy: 0.6953

c:\Users\mujta\anaconda3\envs\churn_analysis\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


175/177 [============================>.] - ETA: 0s - loss: 1.2978 - accuracy: 0.7082
Epoch 2: val_loss improved from 0.58182 to 0.56456, saving model to d:\Customer-Churn-Analysis\Predictive_Modeling/trained_model\best_model.h5
177/177 [==============================] - 1s 3ms/step - loss: 1.2932 - accuracy: 0.7086 - val_loss: 0.5646 - val_accuracy: 0.7644
Epoch 3/50
151/177 [========================>.....] - ETA: 0s - loss: 0.8494 - accuracy: 0.7301
Epoch 3: val_loss improved from 0.56456 to 0.54509, saving model to d:\Customer-Churn-Analysis\Predictive_Modeling/trained_model\best_model.h5
177/177 [==============================] - 0s 3ms/step - loss: 0.8290 - accuracy: 0.7275 - val_loss: 0.5451 - val_accuracy: 0.7644
Epoch 4/50
167/177 [===========================>..] - ETA: 0s - loss: 0.7471 - accuracy: 0.7468
Epoch 4: val_loss improved from 0.54509 to 0.53597, saving model to d:\Customer-Churn-Analysis\Predictive_Modeling/trained_model\best_model.h5
177/177 [=======================

## Saving Results

After training, we save the model architecture, training history, and the best-performing model to the appropriate directories. This allows us to review the model's performance and use the trained model for future predictions without retraining.

The saved files include:
- `ann_architecture.json`: The structure of the trained model.
- `training_results.txt`: The history of training metrics.
- `best_model.h5`: The best model based on validation performance.


## Making Predictions

With the trained model, we now make predictions on the testing data. These predictions will be compared to the actual values in the test dataset to evaluate the model's performance.

The predictions are saved to a CSV file (`predictions.csv`) for further analysis.


## Making Predictions


In [20]:
# Make predictions on the test data
# The predict method will return probabilities, so we convert them to binary class predictions
predictions = (model.predict(X_test) > 0.5).astype("int32").flatten()

# Save the predictions to a CSV file
predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
predictions_path = os.path.join(base_dir, config['predictions_path'])
predictions_df.to_csv(predictions_path, index=False)

print(f"Predictions saved to {predictions_path}")


45/45 [==============================] - 0s 2ms/step
Predictions saved to d:\Customer-Churn-Analysis\Predictive_Modeling/results/predictions.csv


## Running the Tally Function

The tally function is used to compare the model's predictions with the actual values in the test dataset. It checks if the predictions match the actual outcomes and saves the results to a CSV file (`tally_results.csv`).

This step is crucial for validating the model's accuracy and ensuring that the predictions align with the expected results.


## Running the Tally Function


In [21]:
import importlib.util
import os

# Define the path to the PredictionTally.py file
utils_dir = os.path.abspath('../../utils')
prediction_tally_path = os.path.join(utils_dir, 'PredictionTally.py')

spec = importlib.util.spec_from_file_location("PredictionTally", prediction_tally_path)
PredictionTally = importlib.util.module_from_spec(spec)
spec.loader.exec_module(PredictionTally)

# Define the base directory
base_dir = os.path.dirname(os.path.dirname(os.path.abspath('')))

# Now use the function
PredictionTally.run_prediction_tally(base_dir)


Tally results saved to d:\Customer-Churn-Analysis\Predictive_Modeling/results\tally_results.csv
All predictions match the actual values in test data.


## Results Obtained

After running the predictive modeling process, the following key outputs were obtained:

1. **Model Architecture (`ann_architecture.json`)**: This file contains the structure of the ANN model, which includes the configuration of layers and activation functions.
2. **Training Results (`training_results.txt`)**: This file provides insights into the training process, including metrics like accuracy and loss over epochs.
3. **Best Model (`best_model.h5`)**: The best-performing model saved during training, which can be used for future predictions.
4. **Predictions (`predictions.csv`)**: Contains the predicted values for the test dataset.
5. **Tally Results (`tally_results.csv`)**: The results of comparing predictions with actual outcomes, confirming the accuracy of the model.

These results validate the effectiveness of the model and provide a basis for further analysis.


## Summary/Conclusion

In this task, we successfully defined and implemented the architecture of an Artificial Neural Network (ANN) to predict customer churn. The model was trained, validated, and its predictions were tallied against actual outcomes. The process demonstrated the model's capability to learn from historical data and make accurate predictions on new data.

Key takeaways include:
- The ANN model effectively learned from the provided features and made accurate predictions.
- The use of dropout layers and validation helped prevent overfitting, ensuring the model's generalizability.
- The tally function confirmed that the predictions aligned with the actual outcomes, validating the model's accuracy.

These steps lay the foundation for deploying the model in a production environment or refining it further based on additional data and requirements.


## Next Steps

Following the successful definition and training of the ANN model, the next steps involve:

1. **Task 2: Train the Model and Optimize Convergence**: Further fine-tune the model to improve its performance and ensure it converges effectively without overfitting.
2. **Task 3: Predict Customer Churn Based on Critical Attributes**: Use the trained model to predict customer churn and analyze the critical attributes influencing the predictions.
3. **Task 4: Evaluate Model Performance and Analyze Predictions**: Assess the model's performance using various metrics and gain insights into the predictions made by the model.

These steps will further enhance the model's predictive capabilities and provide deeper insights into customer churn behavior.
